In [ ]:
# from pprint import pprint.pprint as pprint
import pprint

Setup
-----------

Run the following series of cells to initialize the required dependancies and 
load the dataset

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai #egg=txtai[graph,pipeline,similarity] datasets ipyplot

In [ ]:
!pip install datasets -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 14.9 MB/s eta 0:00:00


In [ ]:
# downloads and prepares the Flint dataset as a Pandas dataframe
from datasets import load_dataset
import ast
from dateutil import parser

dataset = load_dataset("pnadel/michgovparsed8_16")

temp = dataset['train'].to_pandas()
for col in ['From', 'Sent', 'To', 'Cc', 'Subject', 'Attachment']:
    temp[col] = temp[col].apply(ast.literal_eval)

df = temp.drop(temp.loc[temp.Body == 'nan'].index)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5933 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pnadel___parquet/pnadel--michgovparsed8_16-9af338568c5dfeeb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Index dataset.
#  This would be where you choose what document size you want
rows = df["Body"][200:500]
# rows = df["Body"][:]
# embeddings.index((uid, text, None) for uid, text in enumerate(rows))
rows = rows.array.tolist()

Hugging Face interface setup
---------

In [ ]:
pip install -U sentence-transformers

In [ ]:
# Upload the data and replace the path. Then run the following 2 cells to get the tagged email data for emails 200-500
!unzip /content/97e76ea3-797e-4955-8973-20b8f965f924.zip

Archive:  /content/97e76ea3-797e-4955-8973-20b8f965f924.zip
  inflating: admin.jsonl             


In [ ]:
import json
filename = "/content/admin.jsonl"
with open(filename) as f:
  lines = f.readlines()

labled_data= []
for s in lines:
  result = json.loads(s)
  labled_data.append(result)

# some of the emails may have been put out of order so this fixes that
labled_data.sort(key = lambda x: x['id'])

In [ ]:
# this is the form for the output of the function
# [[query0,[result0,result1,result2]],[query1,[result0,result1,result2]]...]
def run_queries(model_name, queries, rows, similarity,num_results):
  # initializes the array that will contain the return value of the function
  model_storage = []
  for i in range(len(queries)):
    model_storage.append([queries[i],[]])

  model = SentenceTransformer(model_name)
  doc_emb = model.encode(rows)

  for i in range(len(queries)):
    query_emb = model.encode(queries[i])

    # chooses the similarity search. If you want, you can add others
    if (similarity == 'dot'):
      scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    elif (similarity == 'cos'):
      scores = util.cos_sim(query_emb, doc_emb)[0].cpu().tolist()
    else:
      raise ValueError('invalid similarity! Supported arguments: dot, cos')
    
    # selects the num_results number best results for the query by similarity
    doc_score_pairs = list(zip(rows, scores))
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    model_storage[i][1] = doc_score_pairs[0:num_results]

  return model_storage

In [ ]:
# model results is the list produced by run_queries
# rows is the original data onverted to a list
# labled_data is the docanno labled data
def compute_model_score(model_results, rows, labeled_data):
  score = 0
  for x in model_results:  
    # converts between the Doccano document label and the desired semantic search query 
    if x[0] == "public releases":
      query = "public releases (press and state)"
    else:
      query = x[0]
    
    for y in x[1]:
      # if there were no lables for the data automatically fail the test
      if len(labled_data[rows.index(y[0])]["label"]) == 0:
        score-=1
      else:
        # determines if the retrieved document is relevant for the query
        if (query in labeled_data[rows.index(y[0])]["label"][0]):
          score+= 1
        else:
          score-=1 
          
  return score    


In [ ]:
# print(x[1][0][1][0][0]) # <-- gives actual message
#   print(x[1][0][0]) # <-- prints label

In [ ]:
# This is where we see which model performed best on the data
# Runs all of the models on the documents and sees which ones produce emails
# that have tags relevant to the queries. 
# This cell returns the score of each model for selecting relevant emails for queries.
# Note that higher numbers (less negative) are better than smaller ones
from sentence_transformers import SentenceTransformer, util
import pprint

# intimidation is omitted as there is only one instance of it in the dataset
queries = ["taking responsibility", 
          #  "intimidation", 
           "bad judgement because of groupthink", 
           "confusion", 
           "opportunism",
           "public releases"]


# feel free to add other models here for testing
dot_models = ["sentence-transformers/all-mpnet-base-v2",
              "sentence-transformers/all-MiniLM-L12-v2",
              "sentence-transformers/msmarco-bert-base-dot-v5"]

cos_models = ["sentence-transformers/msmarco-distilbert-cos-v5",
              "sentence-transformers/msmarco-distilbert-base-v4"]


# this runs the search queries. It can take a really long time
dot_model_results = []
for x in dot_models:
  dot_model_results.append([x,run_queries(x,queries,rows,'dot',5)])

cos_model_results = []
for x in cos_models:
  cos_model_results.append([x,run_queries(x,queries,rows,'cos',5)])


# scores models based on their searches and outputs the result
all_model_scores = []

for x in dot_model_results:
  all_model_scores.append((x[0], compute_model_score(x[1],rows,labled_data)))

for x in cos_model_results:
  all_model_scores.append((x[0], compute_model_score(x[1],rows,labled_data)))

pprint.pprint(all_model_scores)



When num_results=3 these are the scores
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[('sentence-transformers/all-mpnet-base-v2', -15),
 ('sentence-transformers/all-MiniLM-L12-v2', -13),
 ('sentence-transformers/msmarco-bert-base-dot-v5', -9),
 ('sentence-transformers/msmarco-distilbert-cos-v5', -11),
 ('sentence-transformers/msmarco-distilbert-base-v4', -11)]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 when num_results=5 these are the scores:
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 [('sentence-transformers/all-mpnet-base-v2', -19),
 ('sentence-transformers/all-MiniLM-L12-v2', -21),
 ('sentence-transformers/msmarco-bert-base-dot-v5', -15),
 ('sentence-transformers/msmarco-distilbert-cos-v5', -21),
 ('sentence-transformers/msmarco-distilbert-base-v4', -21)]
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 'sentence-transformers/msmarco-bert-base-dot-v5' performs best in both trials

In [ ]:
  # print(x[1][0][1][0][0]) # <-- gives actual message
  # print(x[1][0][0]) # <-- prints label

In [ ]:
# This is the UI that calls all the other functions and does the actual searches

# feel free to add other queries to this list. However, you should make sure
# that those are supported in the Doccano tags. 
queries = ["taking responsibility", 
           "intimidation", 
           "bad judgement because of groupthink", 
           "confusion", 
           "opportunism",
           "public releases"]


# choose which documents you want to search on
rows2 = df["Body"][:]
rows2 = rows2.array.tolist()

pprint.pprint(run_queries('sentence-transformers/msmarco-bert-base-dot-v5',queries,rows2,'dot',3))

[['taking responsibility',
  [(': Those Responsible for Flint Water Crisis Must Be Held Accountable FOR '
    'IMMEDIATE RELEASE News from the Michigan Democratic Party October 13, '
    '2015 Contact: Paul Kanan, 517.371 .5410; pkanan@michiqandems.com MDP '
    'Statement: Those Responsible for Flint Water Crisis Must Be Held '
    'Accountable LANSING - The Michigan Democratic Party issued the following '
    'statement today, on behalf of Party Chair Brandon Dillon, demanding those '
    'responsible for the Flint water crisis be held accountable and calling on '
    'Governor Snyder to immediately fire former Flint EM, and current EM for '
    'Detroit Public Schools, Darnell Earley: “Accountability in government '
    'means answering to the people you serve, not just cleaning up your '
    'mistakes using their money,” said Brandon Dillon, Chair of the Michigan '
    'Democratic Party. “This catastrophe began with the governor’s appointed '
    'emergency manager, and as much as 

In [ ]:
# queries = ["taking responsibility for filthy water", 
#            "filthy water deception", 
#            "Political pressure menacing", 
#            "groupthink collective incompetance", 
#            "confusion about water cleanliness",
#            "media outrage",
#            "opportunism taking advantage of water"]
# # that last one is artificial. I added it in when I saw a result that fit from a different unrelated query
# print("%-20s %s" % ("Query", "Best Match"))
# print("-" * 50)

# # Run an embeddings search for each query
# for query in queries:
#     # Extract uid of first result
#     # search result format: (uid, score)
#     # note that embeddings.search(query, 1)[x][0] gets the x+1th most similar result
#     uid = embeddings.search(query, 5)[0][0]

#     # Print text
#     print(str(query)+":")
#     pprint.pprint(rows[uid])
#     print()
#     # pprint.pprint("%-20s %s" % (query, rows[uid]))

["taking responsibility", 
           "deception", 
           "intimidation", 
           "bad judgement because of groupthink", 
           "confusion",
           "media outrage",
           "opportunism",
           "boring irrelevant"]